<a href="https://colab.research.google.com/github/srujanperam/dl_1/blob/main/notebooks/Neural_Network_In_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/DeepLearningML/6_gradient_descent/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],
                      df.bought_insurance,test_size=0.2, random_state=25)

In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid',
                       kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 327ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/100
1/1 [==============================] - 0s 8ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/100
1/1 [=========

In [8]:
model.evaluate(X_test_scaled,y_test)


1/1 [==============================] - 0s 124ms/step - loss: 0.6918 - accuracy: 0.5000


[0.6917626261711121, 0.5]

In [9]:
model.predict(X_test_scaled)


array([[0.7743647],
       [0.7252106],
       [0.5345471],
       [0.7428876],
       [0.7775141],
       [0.7957479]], dtype=float32)

In [10]:
y_test


2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [11]:
coef, intercept = model.get_weights()


In [12]:
coef, intercept


(array([[0.905744 ],
        [0.9045079]], dtype=float32), array([-0.09708447], dtype=float32))

In [13]:
def sigmoid(x):
        import math
        return 1 / (1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

In [14]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.7743647160414329

In [15]:
prediction_function(.18, 1)


0.7252106529516228

In [16]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [17]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [20]:
class myNN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [21]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=8000, loss_thresold=0.4631)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:0.7113403233723417
Epoch:50, w1:1.503319554173139, w2:1.108384790367645, bias:-1.2319047301235464, loss:0.5675865113475955
Epoch:100, w1:2.200713131760032, w2:1.2941584023238903, bias:-1.6607009122062801, loss:0.5390680417774752
Epoch:150, w1:2.8495727769689085, w2:1.3696895491572745, bias:-1.986105845859897, loss:0.5176462164249293
Epoch:200, w1:3.443016970881803, w2:1.4042218624465033, bias:-2.2571369883752723, loss:0.5005011269691375
Epoch:250, w1:3.982450494649576, w2:1.4239127329321233, bias:-2.494377365971801, loss:0.48654089537617085
Epoch:300, w1:4.472179522095914, w2:1.4387879865535518, bias:-2.7073878119223727, loss:0.4750814640632793
Epoch:350, w1:4.917245868007633, w2:1.452566078117612, bias:-2.9011763335567657, loss:0.46561475306999006
Epoch:366, w1:5.051047623653048, w2:1.4569794548473884, bias:-2.9596534546250033, loss:0.4629394409588892
Final weights and bias: w1: 5.051047623653048, w2:

In [23]:
coef, intercept

(array([[0.905744 ],
        [0.9045079]], dtype=float32), array([-0.09708447], dtype=float32))

In [22]:
customModel.predict(X_test_scaled)

2     0.705020
10    0.355836
21    0.161599
11    0.477919
14    0.725586
9     0.828987
dtype: float64

In [24]:
model.predict(X_test_scaled)


array([[0.7743647],
       [0.7252106],
       [0.5345471],
       [0.7428876],
       [0.7775141],
       [0.7957479]], dtype=float32)